In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import h5py

from matplotlib.lines import Line2D
from matplotlib.patches import Patch

# set cosmological model
from colossus.cosmology import cosmology
params = {'flat': True, 'H0': 67.77, 'Om0': 0.307, 'Ob0': 0.04825, 'sigma8': 0.8288, 'ns': 0.9611}
cosmology.addCosmology('planck14', params)
cosmo = cosmology.setCosmology('planck14')

In [2]:
# 定义hdf5文件结构读取函数
def prt(name):
    print(name)

In [3]:
filedir_sub = '/Simulations/Magpie/Merger_Tree/trees/M1_zcut7/output/Subfind.0.hdf5'
filedata_sub = h5py.File(filedir_sub, 'r') 
filedata_sub.visit(prt)

Constants
FileInfo
FileInfo/ExpansionFactorAtSnap
FileInfo/FilesReadPerSnap
FileInfo/FirstSnap
FileInfo/LastSnap
FileInfo/NumberOfGroups
FileInfo/SimDir
Header
MergerTree
MergerTree/DescendantID
MergerTree/GalaxyID
MergerTree/LastProgID
MergerTree/TopLeafID
MergerTree/nodeIndex
Subhalo
Subhalo/ApertureMeasurements
Subhalo/ApertureMeasurements/Mass
Subhalo/ApertureMeasurements/Mass/001kpc_BH
Subhalo/ApertureMeasurements/Mass/001kpc_DM
Subhalo/ApertureMeasurements/Mass/001kpc_Gas
Subhalo/ApertureMeasurements/Mass/001kpc_Star
Subhalo/ApertureMeasurements/Mass/003kpc_BH
Subhalo/ApertureMeasurements/Mass/003kpc_DM
Subhalo/ApertureMeasurements/Mass/003kpc_Gas
Subhalo/ApertureMeasurements/Mass/003kpc_Star
Subhalo/ApertureMeasurements/Mass/005kpc_BH
Subhalo/ApertureMeasurements/Mass/005kpc_DM
Subhalo/ApertureMeasurements/Mass/005kpc_Gas
Subhalo/ApertureMeasurements/Mass/005kpc_Star
Subhalo/ApertureMeasurements/Mass/010kpc_BH
Subhalo/ApertureMeasurements/Mass/010kpc_DM
Subhalo/ApertureMeasureme

In [4]:
def find_main_branch_stellar_new(snapNum, GroupNum, SubGroupNum):
    filedir_sub = '/Simulations/Magpie/Merger_Tree/trees/M1_zcut7/output/Subfind.0.hdf5'
    with h5py.File(filedir_sub, 'r') as filedata_sub:
        # print(filedata_sub['Subhalo/SnapNum'])
        index = (filedata_sub['Subhalo/SnapNum'][()]==snapNum) \
                & (filedata_sub['Subhalo/GroupNumber'][()]==GroupNum) \
                & (filedata_sub['Subhalo/SubGroupNumber'][()]==SubGroupNum)
        Topleaf = filedata_sub['MergerTree/TopLeafID'][index]
        ind = np.where(filedata_sub['MergerTree/TopLeafID'][()] == Topleaf[0])[0]
        stellar = filedata_sub['Subhalo/Stars/Mass'][ind]
        snap = filedata_sub['Subhalo/SnapNum'][ind]
        groupnum = filedata_sub['Subhalo/GroupNumber'][ind]
        subgroupnum = filedata_sub['Subhalo/SubGroupNumber'][ind]
        subpos = np.hstack((filedata_sub['Subhalo/CentreOfPotential_x'][ind].reshape(-1,1),\
                            filedata_sub['Subhalo/CentreOfPotential_y'][ind].reshape(-1,1),\
                            filedata_sub['Subhalo/CentreOfPotential_z'][ind].reshape(-1,1)))
        redshift = filedata_sub['Subhalo/Redshift'][ind]
        subhalo_mass = filedata_sub['Subhalo/Mass'][ind]
    return stellar, snap, groupnum, subgroupnum, subpos, redshift, subhalo_mass

In [5]:
# mw_stellar, mw_snap, mw_groupnum, mw_subgroupnum, mw_subpos, mw_redshift, mw_halo = \
#                 find_main_branch_stellar_new(199, 1, 0)

In [ ]:
lmc_stellar, lmc_snap, lmc_groupnum, lmc_subgroupnum, lmc_subpos, lmc_redshift, lmc_halo = \
                find_main_branch_stellar_new(199, 1, 1)

In [17]:
mw_r200_phy.shape

(200,)

In [22]:
mw_groupnum

array([ 9, 11,  3,  8,  5,  7,  6,  7,  6,  7,  6, 14, 18, 18, 15, 12,  2,
        3,  4,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  2,  2,  2,  2,  2,
        2,  2,  1,  2,  2,  2,  2,  2,  3,  3,  2,  3,  3,  1,  1,  1,  2,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  1,  1,  1,  1,  2,
        2,  2,  2,  2,  2,  1,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1], dtype=uint32)

In [55]:
def read_central_r200(simulation_name):
    simulation_num = int(simulation_name[1])
    if simulation_name[3:] == 'zcut7':
        if simulation_num == 4:
            centerIndex = 2
        elif simulation_num == 5:
            centerIndex = 2
        else:
            centerIndex = 1
    elif simulation_name[3:] == '7DM_GAS':
        if simulation_num == 2:
            centerIndex = 2
        elif simulation_num == 4:
            centerIndex = 3
        else:
            centerIndex = 1
    h = 0.6777
    mw_stellar, mw_snap, mw_groupnum, mw_subgroupnum, mw_subpos, mw_redshift, mw_halo = \
                find_main_branch_stellar_new(199, centerIndex, 0)
    snap = mw_snap
    groupnum = mw_groupnum
    r200 = []
    # m200 = []

    filedir = '/Simulations/Magpie/' + simulation_name 
    filename = os.listdir(filedir)
    filename_group = [name for name in filename if name[:6]=='groups' and name[-3:]!='tar']
    Num = np.array([int(filename[7:10]) for filename in filename_group])
    for i in snap[::-1]:
        print(i)
        j = np.where(Num==i)[0][0]
        filedir_group = filedir + '/' + filename_group[j]
        filenamelist_all_group = os.listdir(filedir_group)
        filenamelist_group = [filenamelist_all_group[i] for i in range(len(filenamelist_all_group))\
                            if filenamelist_all_group[i].split('.')[0][:17] == 'eagle_subfind_tab' ]
        R200 = []
        # M200 = []
        for k in range(len(filenamelist_group)):
            filedata_group = h5py.File(filedir_group + '/' + filenamelist_group[k][:30] + '.%0.f.hdf5' % k, 'r')
            catalog = list(filedata_group['FOF'].keys())
            if 'Group_R_Crit200' in catalog:
                R200 += list(filedata_group['FOF/Group_R_Crit200'])
                # M200 += list(filedata_group['FOF/Group_M_Crit200'])
        R200 = np.vstack(R200)
        # M200 = np.vstack(M200)
        if len(R200) < groupnum[snap==i]-1:
            r200.append(np.array([[0]]))
        else:
            r200.append(R200[groupnum[snap==i]-1])
        # m200.append(M200[groupnum[snap==i]-1])
    mw_r200 = np.array([r200[i][0][0] for i in range(len(r200))])[::-1]    # cMpc/h
    mw_r200_phy = mw_r200 * 1/(1+mw_redshift) * 1000 / h   
    return mw_r200_phy[1:][::-1]

In [59]:
simulation_name = 'M3_zcut7'
r200 = read_central_r200(simulation_name)

199
198
197
196
195
194
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
171
170
169
168
167
166
165
164
163
162
161
160
159
158
157
156
155
154
153
152
151
150
149
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
115
114
113
112
111
110
109
108
107
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


In [60]:
r200

array([233.67391041, 233.23022594, 232.7190133 , 232.14497504,
       231.57610121, 230.50362681, 229.65718075, 229.26966724,
       228.94184813, 228.29468631, 227.62800614, 227.03069831,
       226.52619559, 225.98209377, 225.33773779, 224.62427504,
       224.02127268, 223.35324418, 222.91335168, 223.59450636,
       224.03153293, 223.87101633, 222.95588894, 221.65631948,
       220.4591007 , 219.08487962, 217.82296268, 216.93969377,
       215.92119914, 214.76879619, 213.32632659, 211.39618887,
       208.90782923, 205.44921738, 201.31164498, 198.9281519 ,
       196.88792884, 194.79577087, 192.07960209, 189.80866594,
       187.06572908, 184.52470778, 181.3470348 , 177.84642425,
       171.7983439 , 168.22800891, 165.62412965, 163.15072145,
       160.70197461, 157.67718614, 155.5262762 , 153.78311721,
       152.14742871, 150.60256309, 149.17657915, 147.64357607,
       145.66960733, 143.2531222 , 142.36932693, 141.52248068,
       140.47503105, 138.37645011, 137.88423567, 137.87

In [61]:
np.savetxt('data/central_r200_'+ simulation_name, r200)